## Set up dev environment

### Setting up env variables

- Consider changing the details in case using local machine according to your system requirements


In [ ]:
%env DATABASE_HOST=localhost
%env DATABASE_PORT=5432
%env DATABASE_USER=postgres
%env DATABASE_PASS=postgres
%env DATABASE_NAME=demo

env: DATABASE_HOST=localhost
env: DATABASE_PORT=5432
env: DATABASE_USER=postgres
env: DATABASE_PASS=postgres
env: DATABASE_NAME=demo


### Installing Posgresql database
 - use this in case your system do not have posgresql installed
 - Incase you are using colab then you need to use this execution

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"


 * Starting PostgreSQL 12 database server
   ...done.
ALTER ROLE


### Cloning the mathesar repository
- Directly getting the latest db repo from github

In [ ]:
!git clone https://github.com/centerofci/mathesar.git

fatal: destination path 'mathesar' already exists and is not an empty directory.


### Install requirements 
- Install only the nessary requirements


In [ ]:
%pip install "pglast==3.4"
%pip3 install pipreqs
%pipreqs "/content/mathesar/db"
%pip3 install -r "/content/mathesar/db/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Set up dev environment

- Moving the db repo to main environment

In [ ]:
#moving db to main dev environment
%cp -r "mathesar/db" "."

## Start development

### Installing mathesar on the database

> Initial setup



In [ ]:
from db.install import install_mathesar
import os
#installing mathesar

install_mathesar(database_name=os.environ['DATABASE_NAME'],
                 username=os.environ['DATABASE_USER'],
                 password=os.environ['DATABASE_PASS'],
                 hostname=os.environ['DATABASE_HOST'],
                 port=os.environ['DATABASE_PORT'],
                 database_type='',
                 skip_confirm=True
                 )

Created DB is demo.
Installing Mathesar on PostgreSQL database demo at host localhost...


### Getting engine
- Here we create the engine for execution


In [ ]:
from db.engine import create_future_engine
engine=create_future_engine(database=os.environ['DATABASE_NAME'],
                 username=os.environ['DATABASE_USER'],
                 password=os.environ['DATABASE_PASS'],
                 hostname=os.environ['DATABASE_HOST'],
                 port=os.environ['DATABASE_PORT'],)

### Setting schema

In [ ]:
from db.schemas.operations.create import create_schema
schema_name="MYSchema"
schema=create_schema(schema_name,engine,)

### Creating the Table

In [ ]:
from sqlalchemy.sql.sqltypes import INTEGER
from sqlalchemy import VARCHAR, Column, Table, MetaData


def create_test_table(table_name, cols, insert_data, schema, engine):
    table = Table(
        table_name,
        MetaData(bind=engine, schema=schema),
        *cols
    )
    table.create()
    with engine.begin() as conn:
        for data in insert_data:
            conn.execute(table.insert().values(data))
    table.schema=schema
    return table

def create_pizza_table(engine, schema):
    table_name = 'Pizzas18'
    cols = [
        Column('ID',VARCHAR,primary_key=True),
        Column('Pizza', VARCHAR),
        Column('Checkbox', VARCHAR),
        Column('Rating', VARCHAR)
    ]
    insert_data = [
        ("1", 'Pepperoni', 'true', '4.0'),
        ( "2",'Supreme', 'false', '5.0'),
        ( "3",'Hawaiian', 'true', '3.5')
    ]
    return create_test_table(table_name, cols, insert_data, schema, engine)



table=create_pizza_table(engine,schema)

### Simple CRUD application

- creating record into a table
- reading records from table
- updating record in table
- Delete record

#### Create records

In [ ]:
from db.utils import execute_statement

def create_records(records:list,engine=engine)->None:
    execute_statement(engine,table.insert().values(records))


#### Reading records

In [ ]:
from db.records.operations.select import get_records
def read_records(table=table,engine=engine)->list:
    records=get_records(table,engine)
    return records


#### update records

In [ ]:
from db.records.operations.update import update_record as ur
def update_record(record,id,table=table,engine=engine):
    ur(table,engine,id,record)
update_record(("1", 'Mepperoni', 'true', '4.0'),"1")       

#### delete records

In [ ]:
from db.records.operations.delete import bulk_delete_records
to_delete=['5','6']
def delete_records(to_delete_ids:list,table=table,engine=engine)->None:
    bulk_delete_records(table,engine,to_delete_ids)


#### execute custom query


In [ ]:
from db.utils import execute_statement
def custom_query(query):
    rs=execute_statement(engine,query)
    return rs


## Testing custom function

#### creating records

In [ ]:
create_records(
    records=[
        ( "8",'Mexican', 'true', '5.5'),
        ("9", 'Thailand', 'false', '1.5')
             ]
             )
read_records()

[('2', 'Supreme', 'false', '5.0'),
 ('3', 'Hawaiian', 'true', '3.5'),
 ('1', 'Mepperoni', 'true', '4.0'),
 ('8', 'Mexican', 'true', '5.5'),
 ('9', 'Thailand', 'false', '1.5')]

#### reading records

In [ ]:
read_records()

[('2', 'Supreme', 'false', '5.0'),
 ('3', 'Hawaiian', 'true', '3.5'),
 ('1', 'Mepperoni', 'true', '4.0'),
 ('8', 'Mexican', 'true', '5.5'),
 ('9', 'Thailand', 'false', '1.5')]

#### updating records

In [ ]:
to_update=[
        ( "5",'France', 'true', '5.5'),
        ("6", 'Thailand', 'false', '1.5')
             ]
for record in to_update:
    update_record(record,record[0])
read_records()

[('2', 'Supreme', 'false', '5.0'),
 ('3', 'Hawaiian', 'true', '3.5'),
 ('1', 'Mepperoni', 'true', '4.0'),
 ('8', 'Mexican', 'true', '5.5'),
 ('9', 'Thailand', 'false', '1.5')]

#### deleting records

In [ ]:
records_to_delete=['1','2']
delete_records(records_to_delete)
read_records()

[('3', 'Hawaiian', 'true', '3.5'),
 ('8', 'Mexican', 'true', '5.5'),
 ('9', 'Thailand', 'false', '1.5')]

#### custom statement

In [ ]:
statement=table.select().where(table.columns[0] =="3" )
rs=custom_query(statement)
list(rs)

[('3', 'Hawaiian', 'true', '3.5')]